In [2]:
import folium
import json
import pandas as pd
import os
import glob 

aliased_zips = {
    '26507': '26505',
    }

def merge_zips_with_aliases(zip_data):
    for z in aliased_zips:
        if z not in zip_data:
            continue
        az = aliased_zips[z]
        if az not in zip_data:
            zip_data[az] = zip_data[z]
            continue
            
        for key in zip_data[z]:
            if az not in zip_data:
                zip_data[az] = {}
            if key not in zip_data[az]:
                zip_data[az][key] = 0
            zip_data[az][key] += zip_data[z][key]
        del zip_data[z]
    

def update_geojson(geojsonfs, tmpjsonf, zip_data):
    merge_zips_with_aliases(zip_data)

    geozips, tooltips = [], ["ZIP Code"]
    for geojsonf in geojsonfs:
        with open(geojsonf, 'r') as jsonFile:
            data = json.load(jsonFile)    
    
        for i in range(len(data['features'])):
            zipcode = data['features'][i]['properties']['ZCTA5CE10']
            if zipcode in zip_data:
                data['features'][i]['properties']["ZIP Code"] = zipcode
                for key in zip_data[zipcode]:
                    data['features'][i]['properties'][key] = str(zip_data[zipcode][key])
                    data['features'][i]['properties'][key.title()] = str(zip_data[zipcode][key])
                    if key.title() not in tooltips:
                        tooltips.append(key.title())
                geozips.append(data['features'][i])

    new_json = {
        'type': 'FeatureCollection',
        'features': geozips,
        }
    open(tmpjsonf, 'w').write(
        json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
    )
    return tooltips


def map_features(lat, lon, geojsonfs, tmpjsonf, zip_data, feature_name):
    ## zip_data = {zipcode: {feature1: val1, feature2: val2}, ...}
    
    tooltips = update_geojson(geojsonfs, tmpjsonf, zip_data)

    # Make dataframe for records of zip: feature data 
    zip_records = [(k, zip_data[k][feature_name]) for k in zip_data]
    df = pd.DataFrame.from_records(
        zip_records, columns=['zip', feature_name])

    # Map records on top of map, according to provided lat/lon
    m = folium.Map(location=[lat, lon], zoom_start=7, tiles='cartodbpositron')
    choropleth = folium.Choropleth(
        geo_data=tmpjsonf,
        name='choropleth',
        fill_opacity=0.7,
        line_opacity=0.5,
        data=df,
        columns=['zip', feature_name],
        key_on='feature.properties.ZCTA5CE10',
        fill_color='YlOrRd',
        legend_name='{0} by ZIP Code'.format(feature_name.title()), 
        highlight=True,
        reset=True,
    ).add_to(m)

    folium.LayerControl().add_to(m)

    # Add hover tooltips for all data in zip_data
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(tooltips)
    )
    return m

In [5]:
claim_counts_by_zip = [
    ('26505', 216482),  ## Had to change it from 26507, which is not in the shapefiles
    ('26330',  93674),
    ('26101',  48647),
    ('26038',  41812),
    ('26201',  28481),
    ('26651',  11955),
    ('26506',  11029),
    ('26155',  11008),
    ('25271',   7140),
    ('26624',   6663),
    ('26726',   3876),
    ('26301',    621),
    ('26003',    348),
    ('25801',    330),
    ('26537',    324),
    ('25309',    246),
    ('25301',    226),
    ('26062',    193),
    ('26102',    181),
    ('26554',    181),
    ('26287',    164),
    ('25276',    135),
    ('25701',     97),
    ('26041',     82),
]

geojsonfs = glob.glob('geojson/*json')
tmpjsonf = os.path.join('geojson_tmp', 'tmp.json')

feature_name = "Number of Claims"
# Convert tuples into a nice list of dicts
zip_data = {t[0]: {"ZIP Code": t[0], feature_name: t[1]} for t in claim_counts_by_zip}

# Lat / lon for West Virginia
lat = 38.5976
lon = -80.4549

m = map_features(lat, lon, geojsonfs, tmpjsonf, zip_data, feature_name)
m